In [1]:
import os
import pandas as pd
import numpy as np

In [5]:
os.listdir('./ml-latest-small/')

['links.csv', 'tags.csv', 'ratings.csv', 'README.txt', 'movies.csv']

In [7]:
rate_df = pd.read_csv('./ml-latest-small/ratings.csv')
display(rate_df.shape)
rate_df.head(3)

(100836, 4)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224


In [9]:
rate_matrix = rate_df.pivot(index='movieId', columns='userId', values='rating').fillna(0).as_matrix()
display(f'rating_matrix.shape: {rate_matrix.shape}')

'rating_matrix.shape: (9724, 610)'

In [38]:
movie_df = pd.read_csv('./ml-latest-small/movies.csv')
movies = movie_df['title'].tolist()

In [17]:
# df_index = ['movie_{0:04d}'.format(i) for i in range(rate_matrix.shape[0])]
# df_cols = ['user_{0:04d}'.format(i) for i in range(rate_matrix.shape[1])]
# pd.DataFrame(rate_matrix, index=df_index, columns=df_cols)

In [32]:
def reduct_matrix(matrix, k):
    u, s, v = np.linalg.svd(matrix, full_matrices=False)
    s_k = np.array(s)
    s_k[k:] = 0
    sigma_k = np.diag(s_k)
    
    return np.dot(u, np.dot(sigma_k, v))

In [33]:
def predict_ranking(user_index, reducted_matrix, original_matrix, n):
    reducted_vector = reducted_matrix[:, user_index]
    
    # 評価済みのデータは 0 にする。
    filter_vector = original_matrix[:, user_index] == 0
    predicted_vector = reducted_vector * filter_vector
    
    # 上位 n 個のアイテムのインデックスを返す
    recommends = [(i, predicted_vector[i]) for i in np.argsort(predicted_vector)[::-1][:n]]
    return recommends

In [41]:
reducted_matrix = reduct_matrix(rate_matrix, k=30)

In [48]:
def print_ranking(user_ids, items):
    for user_id in user_ids:
        predicted_ranking = predict_ranking(user_id - 1, reducted_matrix, rate_matrix, 10)
        print('User: {}:'.format(user_id))
        for item_id, rating in predicted_ranking:
            # アイテムID, 映画タイトル, 予測した評価値を表示
            print('{}: {} [{}]'.format(item_id, items[item_id + 1], rating))
        print('')

In [49]:
print_ranking([400, 500], movies)

User: 400:
461: Scout, The (1994) [2.534556306096043]
1502: Saving Private Ryan (1998) [2.387344773696831]
97: Taxi Driver (1976) [2.2636905781779193]
1733: American History X (1998) [2.2489795204726066]
2144: American Beauty (1999) [2.1546564104536525]
4131: Star Trek: Nemesis (2002) [2.136197586781076]
827: Reservoir Dogs (1992) [1.9946926027305791]
3136: Necessary Roughness (1991) [1.8029526535437104]
6993: Kung Fu Panda: Secrets of the Furious Five (2008) [1.6700890800050265]
0: Jumanji (1995) [1.5655712300155695]

User: 500:
1153: Austin Powers: International Man of Mystery (1997) [1.2566853628570607]
32: Dead Man Walking (1995) [1.2409056758944261]
862: Monty Python and the Holy Grail (1975) [1.2033834851598724]
898: Princess Bride, The (1987) [1.1742054399447457]
1796: Shakespeare in Love (1998) [1.168476503720313]
2194: Ferris Bueller's Day Off (1986) [1.164134077469736]
1938: Matrix, The (1999) [1.1163722231253959]
1882: Office Space (1999) [1.0818277003025896]
436: Naked (199